#### Import

In [1]:
import easyocr
import os
import torch
import matplotlib.pyplot as plt
import random
import glob
from tqdm import tqdm
import numpy as np
import time
from mmocr_eval import eval_ocr_metric
import re

#### Install packges

In [2]:
# !pip install matplotlib
# !pip install rapidfuzz

In [3]:
torch.cuda.is_available()

False

#### Training

Load model by easyOCR API (https://www.jaided.ai/easyocr/documentation/)

In [4]:
start = time.time()
model = 'crnn_30k_lr_25e-2'
iteration = 'iter_20000'

model_path = os.path.join('trainer', 'saved_models',model)
reader = easyocr.Reader(lang_list = ['en','th'],
                    model_storage_directory = model_path,
                    user_network_directory = os.path.join('my_model','user_network'),
                    recog_network  = iteration,
                    config_path = 'crnn',
                    gpu = True)
# reader = easyocr.Reader(lang_list = ['en','th'],gpu = True )
end = time.time()
print(f'loading model time: {end - start}')

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


--------------------
model_path: trainer/saved_models/crnn_30k_lr_25e-2/iter_20000.pth
config: my_model/user_network/crnn.yaml
--------------------
Transformation: None
FeatureExtraction: VGG
SequenceModeling: BiLSTM
Prediction: CTC
loading model time: 3.4915575981140137


visualise testing with some image

In [5]:
# plt.rcParams['font.family'] = 'TH Sarabun New'
# # plt.rcParams['xtick.labelsize'] = 20.0
# # plt.rcParams['ytick.labelsize'] = 20.0
# columns = 5
# rows = 40
# fig = plt.figure(figsize=(15, 15))
# ax = []
# for i in tqdm(range(columns*rows)):
#     # random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
#     # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
#     #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
#     random_file_name = os.listdir(os.path.join('trainer/all_data/testing/testing'))[i]
#     random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
#     img = plt.imread(random_file_path)
#     result = reader.readtext(random_file_path) 
#     ax.append(fig.add_subplot(rows, columns, i+1) )
#     if len(result) > 0:
#         txt = ''
#         for i in result:
#             txt += (' '+i[1])
#         ax[-1].set_title(f'predict: {txt}')
#     else:
#         ax[-1].set_title('No result')
#     plt.imshow(img)
# fig.tight_layout() 
# plt.savefig(f'result.png')
# # plt.show()

In [6]:
# filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
# label_path = 'trainer/all_data/testing/testing/label.txt'
# with open(label_path, encoding='utf8') as file:
#     label = np.loadtxt(file,dtype=str)
# word_correct = 0

In [7]:

# plt.rcParams['font.family'] = 'TH Sarabun New'
# columns = 5
# rows = 200
# fig, ax = plt.subplots(rows, columns, figsize=(20, 20))
# ax.set_facecolor("white")
# counter = 0
# for r in tqdm(range(rows)):
#     for c in range(columns):
#         file_name = re.split(r'[/\\]',filepaths[counter])[-1]
#         file_path = os.path.join('trainer/all_data/testing/testing',file_name)
#         img = plt.imread(file_path)
#         ax[r, c].axis('off')
#         result = reader.readtext(file_path) 
#         if len(result) > 0:
#             txt = ''
#             for i in result:
#                 txt += (' '+i[1])
#             ax[r, c].set_title(f'{file_name} predict: {txt}')
#         else:
#             ax[r, c].set_title('No result')
#         ax[r, c].imshow(img)
#         counter += 1
# plt.subplots_adjust(top = 20)

testing with all files in dataset

In [9]:
filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))[:20]
label_path = 'trainer/all_data/testing/testing/label.txt'
with open(label_path, encoding='utf8') as file:
    label = np.loadtxt(file,dtype=str)
word_correct = 0
filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))

start = time.time()
# filepaths = filepaths[:10]
pred = []
gt = [] # ground truth
for i in tqdm(range(len(filepaths))):
    label_img = label[i][1]

    result = reader.readtext(filepaths[i], batch_size = 8,detail=0)
    try:
        pred.append(result[0])
        if (len(result) > 1):
            print(label_img, result, len(result))
        gt.append(label_img)
    except:
        pred.append('')

        gt.append(label_img)
end = time.time()
output, false_list = eval_ocr_metric(pred, gt)
print(len(false_list))
for k,v in output.items():
    print(f'{k}: {v}')
print(f'time: {end - start}')

100%|██████████| 20/20 [00:01<00:00, 11.23it/s]

pepsin ['ต', 'โต้'] 2
pred_text:  gt_text: วอห์น
pred_text: Tokenism gt_text: เกรอะ
pred_text:  gt_text: MFC
pred_text: Ekonomisuto gt_text: ของ
pred_text: TULLO` gt_text: รภัจจ์
pred_text: Ranisu gt_text: introduce
pred_text: hypo gt_text: หากิน
pred_text:  gt_text: พิพัฒ
pred_text:  gt_text: Kenyon
pred_text: streaks gt_text: ลัวส์
pred_text: nonexistence gt_text: Uses
pred_text:  gt_text: Herman
pred_text: ท่าน gt_text: ยิ้น
pred_text: เ~ gt_text: แซงจ์
pred_text: จ gt_text: Certre
pred_text: ชเนตท gt_text: Salot
pred_text:  gt_text: ท่อ
pred_text: ปศาจ gt_text: ซี่น
pred_text: ต gt_text: pepsin
pred_text: โฉบ gt_text: experience
20
char_recall: 0.0727
char_precision: 0.0755
word_acc: 0.0
word_acc_ignore_case: 0.0
word_acc_ignore_case_symbol: 0.3
1-N.E.D: 0.3139
time: 1.7830350399017334


testing with LMWN

In [ ]:
plt.rcParams['font.family'] = 'TH Sarabun New'
plt.rcParams['xtick.labelsize'] = 20.0
plt.rcParams['ytick.labelsize'] = 20.0
columns = 2
rows = 10
fig = plt.figure(figsize=(10, 10))
ax = []
for i in range(columns*rows):
    random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/LMWN')),1)[0]
    # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
    #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
    # random_file_name = os.listdir(os.path.join('trainer/all_data/testing/LMWN'))[2]
    random_file_path = os.path.join('trainer/all_data/testing/LMWN',random_file_name)
    img = plt.imread(random_file_path)
    result = reader.readtext(random_file_path,detail=0) 
    ax.append(fig.add_subplot(rows, columns, i+1) )
    if len(result) > 0:
        ax[-1].set_title(f'predict: {result}',fontsize=20)
    else:
        ax[-1].set_title('No result',fontsize=20)
    plt.imshow(img)
fig.tight_layout() 
plt.show()